# -- Exploration and Results for D080428.4 --
## --Threshhold Rainrate -- 
Author: Michael Giansiracusa  
Email: <giansiracumt@ornl.gov>  
DQR info: [D080428.4](http://www.archive.arm.gov/ArchiveServices/DQRService?dqrid=D080428.4) twp/met/C2  
DQR submitted by: Michael T. Ritsche, <mritsche@anl.gov>  
Evaluation date: 19 Sept 2018

This scirpt was written for a group of DQRs which contained the following desription.
### Description
The ORG data is collected via analog signal and not digital. Therefore, we continuously collect data even when no rain is occurring. We also capture small events such as leaves, large bugs, dust, etc blowing through the measurement path. Because this discrete event occurs quickly the value is usually captured for 1 or 2 seconds. This value is then averaged to the minute producing rainrate on the order of .009 mm/hr or less. 

These values are not indicative of rain and should be removed from the record or summing over long periods can bias the climate record. We will reprocess the data to remove these small values.
### Data Info
Raw data staged from hpss: /f1/arm/raw/twpmetC3.00 and from amber: /data/archive/twp/twpmetC3.00/ (amber brd = 192.148.97.127). Processed netCDF data staged from amber: /data/archive/twp/twpmetC3.b1/.

### Conclusions...
This task needs more specific guidelines for how to proceed. Since the mean and standard deviation are calculated in the logger it may be impossible to make these changes so that they are meaningul and correct. The max is never below the threshold so we can't ever simply make all values 0.

In [1]:
## Set input directory for netCDF file inspection
'''
Directory should not contain any data that should not be inspected.
Any folders in the directory should be hidden and begin with a period.
'''
input_dir = "/data/project/0021718_1509993009/D080428.3/datastream/twp/twpmetC3.b1"
print("All data in the following folder will be inspected.\n\t{}".format(input_dir))

All data in the following folder will be inspected.
	/data/project/0021718_1509993009/D080428.3/datastream/twp/twpmetC3.b1


In [2]:
from netCDF4 import Dataset
from glob import glob
import os

In [3]:
files = glob(os.path.join(input_dir, "twpmetC3.b1.*"))
files.sort()
if files:
    for f in files[0:5]:
        print(f)
    print("......")
    for f in files[-5:]:
        print(f)
else:
    print("No files returned from input search:\n\t{}".format(os.path.join(input_dir, "twpmetC3.b1.*")))

/data/project/0021718_1509993009/D080428.3/datastream/twp/twpmetC3.b1/twpmetC3.b1.20040701.000000.cdf
/data/project/0021718_1509993009/D080428.3/datastream/twp/twpmetC3.b1/twpmetC3.b1.20040702.000000.cdf
/data/project/0021718_1509993009/D080428.3/datastream/twp/twpmetC3.b1/twpmetC3.b1.20040703.000000.cdf
/data/project/0021718_1509993009/D080428.3/datastream/twp/twpmetC3.b1/twpmetC3.b1.20040704.000000.cdf
/data/project/0021718_1509993009/D080428.3/datastream/twp/twpmetC3.b1/twpmetC3.b1.20040705.000000.cdf
......
/data/project/0021718_1509993009/D080428.3/datastream/twp/twpmetC3.b1/twpmetC3.b1.20041227.000000.cdf
/data/project/0021718_1509993009/D080428.3/datastream/twp/twpmetC3.b1/twpmetC3.b1.20041228.000000.cdf
/data/project/0021718_1509993009/D080428.3/datastream/twp/twpmetC3.b1/twpmetC3.b1.20041229.000000.cdf
/data/project/0021718_1509993009/D080428.3/datastream/twp/twpmetC3.b1/twpmetC3.b1.20041230.000000.cdf
/data/project/0021718_1509993009/D080428.3/datastream/twp/twpmetC3.b1/twpme

## -- Checking variable names --
The following is a exploration of the variables in the netCDF files for this reprocessing job.

>I am attempting to find the names of the variables that contain rain data and are not quality control variables. 

>I'll do this by checking a equidistant selection of files, within which I am checking each variable name for the substring 'precip' and 'rain'.

In [4]:
for i in range(len(files)):
    if i % 200 == 0:
        test_dataset = Dataset(files[0])
        #print("All variables for file - {}".format(files[0]))
        #print("\t" + str(test_dataset.variables.keys()))
        print("\nPrecip and rain variables for file - {}".format(files[0]))
        for key in test_dataset.variables.keys():
            if ("precip" in key or "rain" in key) and "qc_" not in key:
                    print("\t" + key)


Precip and rain variables for file - /data/project/0021718_1509993009/D080428.3/datastream/twp/twpmetC3.b1/twpmetC3.b1.20040701.000000.cdf
	org_precip_rate_mean
	org_precip_rate_std
	org_precip_rate_max
	org_precip_rate_min
	tbrg_precip_total
	tbrg_precip_count


## -- Checking variable values --
An exploration of variables identified for values below threshhold.
> I'll be checking each variable identified in the previous cell for values that are equal to or below the threshhold mentioned in the dqr report (0.009). 

> I have removed the standard deviation variable from this selection. Although it has values below the threshhold, it is not representative of anomolies in the measurement as a result of quick, discrete events such as leaves, large bugs, dust, etc blowing through the measurement path.

In [5]:
threshhold_value = 0.009

In [ ]:
for f in files:
    value_dict, value_list = {}, []
    print(f)
    dataset = Dataset(f)
    for key in dataset.variables.keys():
        if "precip_rate_mean" in key and ("qc_" not in key and "_std" not in key):
            print("\t" + key)
            for value in dataset.variables[key][:]:
                if value > 0 and value < 0.01:
                    print("\t\t" + str(value))
                    if value <= threshhold_value:
                        print("\t\tmodifying " + str(value))
                        value_list.append(value)
            if value_list:
                value_dict[key] = value_list
    if value_dict:
        print(f)
        for key in value_dict.keys():
            print("\t" + key)
            for value in value_dict[key]:
                print("\t\t" + str(value))
    else:
        print("No values below {} in file - {}".format(threshhold_value, os.path.basename(f)))

## -- Conclusions --
> After running the above cells, it is my conclusion that there are no values in the variables that represent rain rate, for any file in the time range identified in the dqr which are below the threshhold. Because of this, there is nothing to change for this job. 

## -- Confirming Results --
Evaluating raw data to confirm results. 

In [1]:
## Set input directory for raw file inspection
'''
Directory should not contain any data that should not be inspected.
Any folders in the directory should be hidden and begin with a period.
'''
raw_input_dir = "/data/project/0021718_1509993009/D080428.3/collection/twp/twpmetC3.00/.from_local_tars/.raw"
# there was also a .orig and .bad directory that contained files that were in the tar bundles with those substrings in the file names

print("All data in the following folder will be modified.\n\t{}".format(raw_input_dir))

All data in the following folder will be modified.
	/data/project/0021718_1509993009/D080428.3/collection/twp/twpmetC3.00/.from_local_tars/.raw


## -- Data Dictionaries --
This evaluation will use data dictionaries in json format to map raw data columns to variable in the corresponding netCDF files. 

In [2]:
import json
import csv
import os
from glob import glob

In [3]:
# Permanent data dictionaries in $REPROC_HOME/working_data_dictionaries folder on adc machines
# *** used gen_dict_gen.py in ADC toolbox bin folder to map columns ***
data_dict_1 = json.load(open("twpmetC3.00.20040701.unknown.json"))

In [4]:
for key in data_dict_1["data"].keys():
    if "precip" in key:
        print("{} - {}".format(key, data_dict_1["data"][key]))

org_precip_rate_mean - {'column': 4, 'data_type': 'float32', 'dimensions': 'time', 'notes': '', 'attributes': {'long_name': 'ORG precipitation rate mean', 'missing_value': -9999, 'units': 'mm/hr', 'valid_max': 500, 'valid_min': 0}}


In [9]:
raw_files = glob(os.path.join(raw_input_dir, "twpmetC3.00.*"))
raw_files.sort()
print(f"Files in directory: {raw_input_dir}")
if raw_files:
    num_rows = 5
    print(f"\tFirst {str(num_rows)}")
    for f in raw_files[0:num_rows]:
        print(os.path.basename(f))
    print("."*42)
    print(f"\tLast {str(num_rows)}")
    for f in raw_files[-num_rows:]:
        print(os.path.basename(f))
else:
    print("No files returned from input search:\n\t{}".format(os.path.join(input_dir, "twpmetC3.b1.*")))

Files in directory: /data/project/0021718_1509993009/D080428.3/collection/twp/twpmetC3.00/.from_local_tars/.raw
	First 5
twpmetC3.00.20040701.000000.raw.20040701000000.dat
twpmetC3.00.20040701.010000.raw.20040701010000.dat
twpmetC3.00.20040701.020000.raw.20040701020000.dat
twpmetC3.00.20040701.030000.raw.20040701030000.dat
twpmetC3.00.20040701.040000.raw.20040701040000.dat
..........................................
	Last 5
twpmetC3.00.20100630.190000.raw.20100630190000.dat
twpmetC3.00.20100630.200000.raw.20100630200000.dat
twpmetC3.00.20100630.210000.raw.20100630210000.dat
twpmetC3.00.20100630.220000.raw.20100630220000.dat
twpmetC3.00.20100630.230000.raw.20100630230000.dat


In [10]:
# Manually set threshhold value 
raw_threshhold_value = 0.009

# Manually set column to threshhold but this could be auto set from output of earlier cell
"""
for smet files up to smet20040517210300_20040517220201.C3
    3 = mean, 4 = std, 5 = max, 6 = min
for .dat files from 20040519055900.dat forward
    4 = mean, 5 = std, 6 = max, 7 = min
"""
eval_column = [4, 5, 6, 7]

## -- Inspecting Raw Data -- 
Below we inspect the raw data for values that are below the threshold value

In [ ]:
for f in raw_files:
    value_dict, value_list = {}, []
    #print(f)
    with open(f) as open_raw_file:
        csv_file_reader = csv.reader(open_raw_file)
        for line in csv_file_reader:
            #print("\tcolumn = {}".format(eval_column))
            value = eval(line[3])
            if line[0] == "1" and (value > 0 and value < raw_threshhold_value):
                value_list.append(value)
                #print("\t\t" + value)
        if value_list:
            key = "org_rate_precip_mean".format(5)
            value_dict[key] = value_list
    if value_dict:
        print(os.path.basename(f))
        for key in value_dict.keys():
            print("\t" + key)
            for value in value_dict[key]:
                print("\t\t" + str(value))
    else:
        #print("No values below {} in file - {}".format(raw_threshhold_value, os.path.basename(f)))
        pass

## -- Revised Conclusion --
Identified values in raw files that are below threshhold value. 

## -- Threshholding Raw Files --
Threshhold the values identified in the previous cell. 

In [15]:
output_directory = "/data/project/0021718_1509993009/D080428.3/collection/twp/twpmetC3.00"
for f in raw_files:
#     print(os.path.basename(f))
    modified_lines = []
    with open(f) as open_raw_file:
        csv_file_reader = csv.reader(open_raw_file)
        for i, line in enumerate(csv_file_reader):
            #print("\tcolumn = {}".format(eval_column))
            max_value = eval(line[eval_column[2]])
            if line[0] == "1":
                if max_value > 0 and max_value < raw_threshhold_value:
                    line[eval_column[0]] = "0"
                    line[eval_column[1]] = "0"
                    line[eval_column[2]] = "0"
                    line[eval_column[3]] = "0"
                    print("{} : {},value {},{},{},{} --> 0".format(os.path.basename(f), str(line[1:4]), mean_value, sd_value, max_value, min_value))
            modified_lines.append(line)
    output_file = os.path.join(output_directory, os.path.basename(f))
    with open(output_file, 'w') as open_output_file:
        csv_file_writer = csv.writer(open_output_file)
        csv_file_writer.writerows(modified_lines)

## -- Post Modification --
After the raw files are modified I ran the `strip_arm_names` script to remove the arm name from the file before ingest.
The effect of which removed the datastream, date, and raw substrings from the file name, turning a file like twpmetC3.00.20040723.020000.raw.20040723020000.dat into 20040723020000.dat and then the files are ingested using the command `met_ingest -d twp -f C3 -DR` so that the cdf files can be compared. NetCDF files compared using script below for small subset of the data and using ncreview for the full data set. Durring the ingest, the -F flag was used to bypass overlapping time records and the resulting ncreview is at the following link: https://engineering.arm.gov/ncreview/?/data/project/0021718_1509993009/post_processing/D080428.3/ncr_twpmetC3.b1.2/

## -- Checking netCDF Files --
I'll be running through the variables I think changed in the netCDF files and checking them for the modifications. I used a DiffMerge to compare the raw files and verified that they changed but when I run ncreview on the netCDF files, it shows no change. My theory is that the change is so small that ncreview isn't showing it but it does exist in the netCDF files. 

In [ ]:
import os
from netCDF4 import Dataset
from glob import glob

orig_dir = "/data/archive/twp/twpmetC3.b1"
mod_dir = "/data/project/0021718_1509993009/D080428.3/datastream/twp/twpmetC3.b1"
'''This part is just a conjunction of 3 things. 
First, use os to create a generalized path name,
second, use glob to get a list of regex matches in that path, 
third, strip the path and just get the base filename.'''
fnames = [os.path.basename(f) for f in glob(os.path.join(mod_dir, "*.cdf"))]

fnames.sort()

for fname in fnames:
    print(f"Inspecting - {fname}")
    do = Dataset(os.path.join(orig_dir, fname))
    dm = Dataset(os.path.join(mod_dir, fname))

    """ Get the variables in the modified file that have the substing precip in them and don't have 
    the substrings qc or std. This effectively gets all rain variables that aren't quality control
    variables and aren't standard deviation ones. """
    vars = [v for v in dm.variables.keys() if ("precip" in v and "qc" not in v and "std" not in v)]

    for var in vars:
        print(f"\t...{var}")
        """ Zip the lists together. Both will get truncated to the size of the smaller one. :-("""
        for x, y in zip(do.variables[var][:], dm.variables[var][:]):
#                 print(x, "--", y)
            # extra prints for alternative comparison functionality
            if x > 0 or y > 0:
                if x != y:
                    print("\t\t{} != {}".format(x, y), end="")
                else:
#                         print("\t{} == {}".format(x, y))
                    pass

## -- Visual comparison --
Comparison of raw files done on local machine using DiffMerge.  
![raw files](files/raw_comparison.png)  
Snapshot of output from last cell which compares the cdf files  
![cdf files](files/cdf_comparison.png)  
From the infomation above I have concluded that (for at least some of the files but I suspect all of them) the cdf files were thresholded before or durring ingest. I checked hpss and these cdf files are all v0 so they haven't been rearchived. This means that changing the raw files will have no effect on the cdf files. Therefore, I'll change the raw files, double check that the cdf files didn't change then rearchive only the changed files. 